In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('bike-sharing-daily.csv')

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
data.drop(['instant','casual', 'registered'], axis=1, inplace= True)

In [ ]:
data.dteday = pd.to_datetime(data.dteday, format = '%m/%d/%Y')

In [ ]:
data.index = pd.DatetimeIndex(data.dteday)

In [ ]:
data['cnt'].asfreq('W').plot(linewidth=3)
plt.title('Bike Usage Per Week')
plt.xlabel('Week')
plt.ylabel('Bike Rental')

In [ ]:
data['cnt'].asfreq('M').plot(linewidth=3)
plt.title('Bike Usage Per Month')
plt.xlabel('Month')
plt.ylabel('Bike Rental')

In [ ]:
data['cnt'].asfreq('Q').plot(linewidth=3)
plt.title('Bike Usage Per Quarter')
plt.xlabel('Quarter')
plt.ylabel('Bike Rental')

In [ ]:
sns.pairplot(data)

In [ ]:
X_numerical = data[['temp', 'hum', 'windspeed','cnt']]

In [ ]:
X_numerical

In [ ]:
sns.pairplot(X_numerical)

In [ ]:
sns.heatmap(X_numerical.corr(), annot=True)

In [ ]:
X_categorical = data[['season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit']]

In [ ]:
X_categorical

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
X_categorical = ohe.fit_transform(X_categorical).toarray()

In [ ]:
X_categorical

In [ ]:
X_categorical.shape

In [ ]:
X_cat = pd.DataFrame(X_categorical)

In [ ]:
X_num = X_numerical.reset_index()

In [ ]:
X_num

In [ ]:
X_all = pd.concat([X_cat,X_num], axis=1)

In [ ]:
X_all

In [ ]:
X_all.drop('dteday', axis=1, inplace = True)

In [ ]:
X_all

In [ ]:
X = X_all.iloc[:, :-1].values
y = X_all.iloc[:, -1:].values

In [ ]:
X.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
y = scaler.fit_transform(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=100, activation='relu', input_shape=(35,)))
model.add(tf.keras.layers.Dense(units=100, activation='relu'))
model.add(tf.keras.layers.Dense(units=100, activation='relu'))
model.add(tf.keras.layers.Dense(units=1, activation='linear'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'Adam', loss='mean_squared_error')

In [ ]:
epochs_hist = model.fit(X_train, y_train[:,0], epochs = 20, batch_size=50, validation_split = 0.2)

In [ ]:
epochs_hist.history.keys()

In [ ]:
plt.plot(epochs_hist.history['loss'])
plt.plot(epochs_hist.history['val_loss'])
plt.title('Model Loss Progress During Training')
plt.xlabel('Epochs')
plt.ylabel('L2 Error')

In [ ]:
y_predict = model.predict(X_test)

In [ ]:
plt.plot(y_test, y_predict, '^', color='r')
plt.xlabel('True Values')
plt.ylabel('Predicted Values')

In [ ]:
y_predict_orig = scaler.inverse_transform(y_predict)
y_test_orig = scaler.inverse_transform(y_test)

In [ ]:
plt.plot(y_test_orig, y_predict_orig, '^', color='r')
plt.xlabel('True Values')
plt.ylabel('Predicted Values')

In [ ]:
k = X_test.shape[1]

In [ ]:
k

In [ ]:
n = len(X_test)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

RMSE = float(format(np.sqrt(mean_squared_error(y_test_orig, y_predict_orig)), '0.3f'))

In [ ]:
RMSE

In [ ]:
MSE = float(format(mean_squared_error(y_test_orig, y_predict_orig), '0.3f'))

In [ ]:
MAE = float(format(mean_absolute_error(y_test_orig, y_predict_orig), '0.3f'))

In [ ]:
MAE

In [ ]:
r2 = r2_score(y_test_orig,y_predict_orig)

In [ ]:
r2

In [ ]:
adj_r2 = 1 - (1-r2)*(n-1)/(n-k-1)

In [ ]:
adj_r2